In [1]:
!pip install langchain langchain-core langchain-community pydantic

In [2]:
!pip install -U ddgs

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests


In [ ]:
HUGGINGFACEHUB_API_TOKEN = "hf_xxxxxxxxxxxxx"

In [53]:
@tool
def multiply(a:int, b:int) -> int:
    """given 2 numbers a and b this tool returns their product"""
    return a * b

In [54]:
llm = HuggingFaceEndpoint(
    repo_id = "deepseek-ai/DeepSeek-V3.2",
    huggingfacehub_api_token= HUGGINGFACEHUB_API_TOKEN
)
model = ChatHuggingFace(llm=llm)

In [55]:
llm_with_tools = model.bind_tools([multiply])

In [56]:
llm_with_tools.invoke('Hii how are you?')

AIMessage(content="I'm doing great, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 502, 'total_tokens': 518}, 'model_name': 'deepseek-ai/DeepSeek-V3.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c23f5-d1d8-79a2-bd1b-5ae3a09ce4d9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 502, 'output_tokens': 16, 'total_tokens': 518})

In [57]:
querry = HumanMessage('can you multiply 3 with 1000')

In [58]:
message = [querry]

In [59]:
message

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [60]:
result = llm_with_tools.invoke(message)

In [61]:
message.append(result)

In [62]:
arguments = result.tool_calls[0]

In [63]:
tool_result = multiply.invoke(arguments)

In [64]:
message.append(tool_result)

In [65]:
llm_with_tools.invoke(message).content

"I see you've provided the result of the multiplication: 3000. That's correct! 3 multiplied by 1000 equals 3000.\n\nIs there anything else I can help you with?"

In [75]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool

def get_conversion_factor(base_currency:str, target_currency:str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/dcb15756dfdd1704a61c91af/pair/{base_currency}/{target_currency}'
    
    response = requests.get(url)
    
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate
    

In [67]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1770076802,
 'time_last_update_utc': 'Tue, 03 Feb 2026 00:00:02 +0000',
 'time_next_update_unix': 1770163202,
 'time_next_update_utc': 'Wed, 04 Feb 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 91.524}

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id = "deepseek-ai/DeepSeek-V3.2",
    huggingfacehub_api_token= "hf_xxxxxxxxxxxxxxx"
)

model = ChatHuggingFace(llm=llm)

In [76]:
llm_with_tools = model.bind_tools([get_conversion_factor, convert])

In [77]:
messages = [HumanMessage('what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')]

In [78]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [79]:
ai_message = llm_with_tools.invoke(messages)

In [80]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'fa7c6e11cb564aefae6cc181a315cb44',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'eb1686235129487fa74a01ffd53a65e8',
  'type': 'tool_call'}]

In [81]:
messages.append(ai_message)

In [82]:
import json

for tool_call in ai_message.tool_calls:
    
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
        
    if tool_call['name'] == 'convert':
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [83]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'fa7c6e11cb564aefae6cc181a315cb44', 'type': 'function'}, {'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert', 'description': None}, 'id': 'eb1686235129487fa74a01ffd53a65e8', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 633, 'total_tokens': 710}, 'model_name': 'deepseek-ai/DeepSeek-V3.2', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c241b-3cb6-7af2-a8ea-c1ffda0c4d86-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'fa7c6e11cb564aefae6cc1

In [84]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 91.524. This means 1 USD equals 91.524 INR.\n\nTherefore, 10 USD is equivalent to 915.24 INR.'